<a href="https://colab.research.google.com/github/ken011528/hkjc-scrapy/blob/main/hkjc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import selenium
from  selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
import time
import datetime
import shlex
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities 
from selenium.webdriver.support.ui import  WebDriverWait
from selenium.webdriver.support import  expected_conditions as EC
from selenium.webdriver.common.by import By


pd.options.mode.chained_assignment = None

     |████████████████████████████████| 911kB 2.7MB/s 
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
G

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
browser = webdriver.Chrome(
    options = options,
)
h_browser = webdriver.Chrome(
    options = options,
)
p_browser = webdriver.Chrome(
    options = options,
)


In [ ]:
url = 'https://racing.hkjc.com/racing/info/meeting/Results/Chinese/Local/'
browser.get(url)

In [ ]:
def get_race_date():
    racedate = browser.find_element_by_xpath('html/body/div').text.split('\n')[4:]
    racedate = [x.strip() for x in racedate]
    race = [str(x) for x in racedate if len(x) ==10 and x.count('/') == 2]
    return [shlex.split(str(datetime.datetime.strptime(x, '%d/%m/%Y')))[0].replace('-','/') for x in race]
time.sleep(0.5)
r = get_race_date()

In [ ]:
def get__single__race(racedate, racehouse, raceno):
    
    url = 'https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?' + 'RaceDate=' + racedate + '&Racecourse='+ racehouse + '&RaceNo=' + raceno
    print(url)
    browser.get(url)
    time.sleep(6)    

    done = False
    while not done:
      try:
        s = browser.find_element_by_xpath('html/body/div').text
       
        
        done = True
      except ValueError:
        print('error')
        browser.get(url)
        s = browser.find_element_by_xpath('html/body/div').text
        
        time.sleep(6)
    
    done = False
    
    while not done:
      try:
        sh = browser.find_element_by_class_name('performance').find_element_by_tag_name('thead').text
        done = True
      except :
        print('error')

        browser.get(url)
        sh = browser.find_element_by_class_name('performance').find_element_by_tag_name('thead').text

        time.sleep(6)

    field__length = browser.find_element_by_class_name('race_tab').find_element_by_class_name('f_fs13').text.split('\n')[0].split(' - ')[1]
     


    name = list(map(lambda x : x.replace('\n',''),sh.split(' ')))
    race_detail = browser.find_elements_by_class_name('fontFam')[1].find_elements_by_tag_name('tr')
    result = pd.DataFrame()
    # print(race_detail)
    for i in race_detail:   
        r = {}
        r['date'] = racedate
        r['raceno'] = raceno
        r['place'] = s[s.index('賽事日期'):].split('\n')[0].split(':')[-1].split(' ')[-1]
        r['length'] = field__length
        # r['length'] = s[s.index('班') + 1:].split('-')[1].strip()
        r['field'] = s[s.index('場地狀況'):s.index('賽道')].split('\n')[0].split(':')[-1].strip()
        r['field_detail'] = s[s.index('賽道'):s.index('時間')].split('\n')[0].split(':')[-1].strip()
        for j in range(len(name)):
                r[name[j]] = i.find_elements_by_tag_name('td')[j].text
        for j in i.find_elements_by_tag_name('a'):
            if 'HorseId' in j.get_attribute('href'):
                id_ = j.get_attribute('href').split('HorseId=')[-1]
                url = 'https://racing.hkjc.com/racing/information/chinese/Horse/Performance.aspx?HorseId=' + id_
                
                try:
                    horse__table = get__horse__lineage(url)
                except:

                      url = 'https://racing.hkjc.com/racing/information/chinese/Horse/OtherHorse.aspx?HorseId=' + id_


                      horse__table = get__horse__lineage(url)

                      # horse__table = pd.DataFrame(np.zeros((1,12)))

                try:
                    horse__history = get__horse__pass__result(url)
                except:      
                    # horse__history = pd.DataFrame(np.zeros((1,287)))
                    horse__history = get__horse__pass__result__retired(url)
                    
                d = pd.concat([horse__table,horse__history], axis = 1)
                # print(url)
                # display(d)
        r = pd.concat([pd.DataFrame(r, index = [0]),d], axis = 1)
        if len(result) != 0:
          r.columns = result.columns



        result = pd.concat([result, r])
    return result.dropna(axis = 0)

In [ ]:
def get__horse__lineage(url):
    h_browser.get(url)
    try:
      WebDriverWait(h_browser,10).until(
          EC.presence_of_element_located((By.CLASS_NAME,'table_top_right'))
      )
    except ValueError:
      pass

    race__data = h_browser.find_element_by_class_name('table_top_right').text
    detail = shlex.split(race__data[race__data.index(':')+1:race__data.index('\n')])
    champ = race__data[race__data.index('冠'):race__data.index('總出賽次數') + 20]
    horse = champ.split(':')[-1].strip().split('\n')[0]
    if len(detail) != 1:
      race__horse__data = {'place': detail[0], 'year': detail[-1], 'champ': horse.split('-')[0],'sliver':horse.split('-')[1],'bronze':horse.split('-')[2],'total__race':horse.split('-')[-1]}
    else:
      race__horse__data = {'place': detail[0], 'year': 0, 'champ': horse.split('-')[0],'sliver':horse.split('-')[1],'bronze':horse.split('-')[2],'total__race':horse.split('-')[-1]}

    ppl__data = h_browser.find_elements_by_class_name('table_top_right')[1].text
    d__ppl = {'trainner': 
        ppl__data[ppl__data.index(':') + 1:ppl__data.index('馬主')].strip() ,
     'owner':
        ppl__data[ppl__data.index('馬主'):ppl__data.index('評')].strip().split(':')[-1].strip().split('\n')[0],
     
     'father':
         ppl__data[ppl__data.index('父'):ppl__data.index('母')].strip().split(':')[-1] ,
    'mother':
         ppl__data[ppl__data.index('母'):ppl__data.index('外')].strip().split(':')[-1] ,
    }
    try:
        d__ppl['current__rate'] = ppl__data[ppl__data.index('現'):ppl__data.index('季')].strip().split(':')[-1] 
    except :
         d__ppl['current__rate'] = ppl__data[ppl__data.index('最'):ppl__data.index('最') + 10].strip().split(':')[-1].strip()
    try:
        d__ppl['before__rate'] = ppl__data[ppl__data.index('季'):ppl__data.index('父')].strip().split(':')[-1] 
    
    except:
         d__ppl['before__rate'] = 0

    return pd.DataFrame(pd.concat([pd.Series(race__horse__data), pd.Series(d__ppl)])).transpose()
    
def get__field__status(input_):
    d = pd.DataFrame()
    data = pd.DataFrame()
    num_ = 0
    weed = ['快地','好地至快地','大爛地','軟地','黏至軟地','黏地','好至黏地','好地']
    weather = ['全-濕快地','全-好地','全-濕慢地','全-例常灑水','全-快地','全-慢地','全-受天雨影響']
    
    detail = input_[input_.index('場地成績'):input_.index('路程成績')]

    
    for idx_i,i in enumerate(detail):
        if '草地' in i or '全天候跑道'in i:
            key = i
            
            if len(d)!=0:
                data = pd.concat([data,d], axis = 1)
                
            dict_ = {}
            # dict_['place'] = key
            d = pd.DataFrame(dict_, index= [0])
            num = 0
            num_ = 0

            for j in weed:
                dict_ = {}
                dict_['status' + str(num)] = j
                dict_['total'+ str(num)] = 0
                dict_['champ'+ str(num)] = 0
                dict_['sliver'+ str(num)] = 0
                dict_['brozen'+ str(num)] = 0
                dict_['out'+ str(num)] = 0
                d = pd.concat([d, pd.DataFrame(dict_, index = [0])],axis = 1)
                num += 1

            for j in weather :
                dict_ = {}
                dict_['status'+ str(num)] = j
                dict_['total'+ str(num)] = 0
                dict_['champ'+ str(num)] = 0
                dict_['sliver'+ str(num)] = 0
                dict_['brozen'+ str(num)] = 0
                dict_['out' + str(num)] = 0
                d = pd.concat([d, pd.DataFrame(dict_, index = [0])],axis = 1)
                num += 1
        if i in weed or '全' + i in weather:

            if key =='全天候':
                name = '全-' + i
            else:
                name = i
            num_ = 1
        if num_ != 0:
            
            if num_ != 1:
                for j in d.columns:
                    if 'status' in j:
                        for k  in d.iloc[:,d.columns.get_loc(j)]:
                            if k == name:
                                idx  = d.columns.get_loc(j)
                
                d.iloc[:,idx + num_ - 1] = i
                

                num_ += 1
            else:
                num_ += 1
        if idx_i == len(detail)+1-2 and len(data) == 0:
            data = pd.concat([data,d], axis = 1)
    return data

def get__horse__pass__result(url):

        h_browser.get(url)
        time.sleep(6)
        input_ = [i.text for i in h_browser.find_element_by_class_name('horseperformance').find_elements_by_tag_name('td')]
        status = get__field__status(input_)
        # print(input_)
        table = h_browser.find_element_by_class_name('horseperformance').find_element_by_tag_name('tbody').text
        
        list_ = range(1000, 2400 + 1, 200)
        data = pd.DataFrame()
        place_list = ['沙田草地','沙田全天候跑道','從化草地','跑馬地草地']
        idx= 0
        for i in place_list:
            key = i
            p = pd.DataFrame()
            dict_ = {}
            dict_['place'] = key
            for j_idx,j in enumerate(list_):
                dict_['length' + str(idx)] = j
                dict_['total-length' + str(idx)] = 0
                dict_['champ-length' + str(idx)] = 0
                dict_['sliver-length' + str(idx)] = 0
                dict_['brozen-length' + str(idx)] = 0
                dict_['out-length' + str(idx)] = 0
                idx += 1
            data = pd.concat([data,pd.DataFrame(dict_,index = [0])],axis = 1)

        place_index = np.where(data.columns.get_loc('place') == True)

        for i in table[table.index('路程成績'):table.index('歷季成績')].split('\n')[1:]:
            try :
                float(i.split(' ')[0].split('米')[0])
                for items  in  place_index:
                    num = items[1] - items[0]
                    for idx in items:
                        if data.iloc[:,idx][0] == key:

                            for idx_j,j in enumerate(data.iloc[:,idx:idx + num].columns):
                                if 'length' in j and '-' not in j:

                                    if data.loc[:,j][0] == i.split(' ')[0]:
                                        idx_ = idx_j + idx
                                        print(idx_)
                                        for k in i.split(' '):
                                            data.iloc[:, idx_][0] = k
                                            idx_ += 1
            except:
                if '總成績' not in i and len(i) != 0:
                    key = i.split(' ')[0]
                    for items  in place_index:
                        num = items[1] - items[0]
                        for idx in items:
                            if data.iloc[:,idx][0] == key:
                                for idx_j,j in enumerate(data.iloc[:,idx:idx + num].columns):
                                    if 'length' in j and '-' not in j:
                                        if data.loc[:,j][0] == int(i.split(' ')[1].split('米')[0].strip()):
                                            idx_ = idx_j + idx
                                            for k in i.split(' ')[1:]:
                                                data.iloc[:, idx_][0] = k
                                                idx_ += 1



        return pd.concat([status,data], axis = 1)





In [ ]:
def get__horse__pass__result__retired(url):

  h_browser.get(url)


  do = False
  while not do:    
    try:
      tr = h_browser.find_element_by_class_name('bigborder').find_elements_by_tag_name('tr')
      do = True
    except ValueError :
      pass
  # time.sleep(6)
  # place	status0	total0	champ0	sliver0	brozen0	out0
  d = pd.DataFrame()
  col = []
  for idx,i in enumerate(tr):
    if idx == 0:
        for j in i.find_elements_by_tag_name('td'):
          col.append(j.text)
        col = [j.replace('\n','') for j in col]
    if i.get_attribute('bgcolor'):
      race = []
      for j in i.find_elements_by_tag_name('td'):
        race.append(j.text)
      d = pd.concat([d, pd.DataFrame(race, columns = [0]).transpose()], axis = 0)
  d = d[d.columns[:-2]]
  # print(d)
  d.columns = col[:-1]
  d.iloc[:,3] = d.iloc[:,3].map(lambda x: x.split('"')[0])
  d['status'] = d.iloc[:,3].apply(lambda x: '草地' if '草地' in str(x) else'全天候')
  d.iloc[:,1] = d.iloc[:,1].apply(lambda x: x if check__digit(x) else 0)
  d.iloc[:,1] = d.iloc[:,1].apply(lambda x: x if int(x) >0 and int(x)<=3 else 0)

  data = pd.DataFrame()
  t = pd.DataFrame(d.groupby(['status', '場地狀況','名次']).count().iloc[:,0]).reset_index().merge(pd.DataFrame(d.groupby(['status', '場地狀況']).count())['名次'].reset_index().rename(columns = {'名次':'Total'}), on = ['status', '場地狀況']).groupby(['status', '場地狀況'])
  d_try = pd.DataFrame()
  num = 0
  num_ = 0
  weed = ['快地','好地至快地','大爛地','軟地','黏至軟地','黏地','好至黏地','好地']
  weather = ['全-濕快地','全-好地','全-濕慢地','全-例常灑水','全-快地','全-慢地','全-受天雨影響']



  for j in weed:
        dict_ = {}
        dict_['status' + str(num)] = j
        dict_['total'+ str(num)] = 0
        dict_['champ'+ str(num)] = 0
        dict_['sliver'+ str(num)] = 0
        dict_['brozen'+ str(num)] = 0
        dict_['out'+ str(num)] = 0
        d_try = pd.concat([d_try, pd.DataFrame(dict_, index = [0])],axis = 1)
        num += 1

  for j in weather :
      dict_ = {}
      dict_['status'+ str(num)] = j
      dict_['total'+ str(num)] = 0
      dict_['champ'+ str(num)] = 0
      dict_['sliver'+ str(num)] = 0
      dict_['brozen'+ str(num)] = 0
      dict_['out' + str(num)] = 0
      d_try = pd.concat([d_try, pd.DataFrame(dict_, index = [0])],axis = 1)
      num += 1


  for idx, kv in enumerate(t):
    d_ = {}
    dict_ = pd.DataFrame()
    k,v = kv
    # print(k[1])
    if k[1] == '好':
      str_ = '好地'
    elif k[1] == '好/快':
      str_ = '好地至快地' 
    elif k[1] == '好/黏':
      str_ = '好至黏地'
    elif k[1] == '爛':
      str_ = '大爛地'
    elif k[1] == '黏/軟':
      str_ = '黏至軟地'
    elif k[1] == '黏':
      str_ = '黏地'
    elif '濕' in k[1] and '快' in k[1]:
      str_ = '濕快地'
    elif '濕' in k[1] and '慢' in k[1]:
      str_ = '濕慢地'
    elif '例' in k[1] or '常' in k[1] or '灑' in k[1] or '水' in k[1]:
      str_ = '例常灑水'
    else:
      str_ = '受天雨影響'
    if '全' in k[0]:
      str_ = '全-' + str_
    
    for idx_, col in enumerate(d_try.columns):
      if 'status' in col:
        # print(d_try[k])
        if d_try[col].iloc[0]  == str_:
          for j in range(len(v)):
            d_try.iloc[:,idx_ + 1] = v['Total'].iloc[j]
            # d_try['Total' + str(idx)] = v['Total'].iloc[j]
            if v['名次'].iloc[j] == '01':
              d_try.iloc[:, idx_ + 2] = v['場次'].iloc[j]
            elif v['名次'].iloc[j] == '02':
              d_try.iloc[:,idx_ + 3] = v['場次'].iloc[j]
            elif v['名次'].iloc[j] == '03':
              d_try.iloc[:,idx_ + 4]= v['場次'].iloc[j]
            else:
            
              d_try.iloc[:,idx_ + 5] = v['場次'].iloc[j]
  temp = d.groupby(['馬場/跑道/賽道', '途程', '名次']).count().iloc[:,0].reset_index().merge(pd.DataFrame(d.groupby(['馬場/跑道/賽道','途程']).count()).reset_index().rename(columns = {'名次':'Total'})[['馬場/跑道/賽道','途程','Total']], on = ['馬場/跑道/賽道', '途程'])
  list_ = range(1000, 2400 + 1, 200)
  data_ = pd.DataFrame()
  place_list = ['沙田草地','沙田全天候跑道','從化草地','跑馬地草地']
  idx= 0
  for i in place_list:
      key = i
      p = pd.DataFrame()
      dict_ = {}
      dict_['place'] = key
      for j_idx,j in enumerate(list_):
          dict_['length' + str(idx)] = j
          dict_['total-length' + str(idx)] = 0
          dict_['champ-length' + str(idx)] = 0
          dict_['sliver-length' + str(idx)] = 0
          dict_['brozen-length' + str(idx)] = 0
          dict_['out-length' + str(idx)] = 0
          idx += 1
      data_ = pd.concat([data_,pd.DataFrame(dict_,index = [0])],axis = 1)

  col_list = []
  for col in data_.columns:
    if 'length' in col and '-' not in col:
      col_list.append(col)

  place_index = np.where(data_.columns.get_loc('place') == True)
  p_dict = {}
  for p in place_index[0]:
    p_dict.update({data_.iloc[:,p].iloc[0]: p})
  diff = abs(sorted(p_dict.items(), key = lambda item:item[1])[-2][-1] - sorted(p_dict.items(), key = lambda item:item[1])[-1][-1])
  for k,v in temp.groupby('馬場/跑道/賽道'):
    if '全天候' in k:
      k = '沙田全天候跑道'
    idx = p_dict[k]
    a = pd.DataFrame(data_.iloc[:, idx:idx + diff].iloc[0]).transpose()    

    for col in a.columns:
      if 'length' in col:
        for j in range(len(v)):
          if v['途程'].iloc[j] == str(a[col].iloc[0]):
            data_.iloc[:,data_.columns.get_loc(col) + 1] = v['Total'].iloc[j]
            if v['名次'].iloc[j] == '01':
              data_.iloc[:,data_.columns.get_loc(col) + 2] = v['場次'].iloc[j]
            elif v['名次'].iloc[j] == '02':
              data_.iloc[:,data_.columns.get_loc(col) + 3] = v['場次'].iloc[j]
            elif v['名次'].iloc[j] == '03':
              data_.iloc[:,data_.columns.get_loc(col) + 4] = v['場次'].iloc[j]
            elif v['名次'].iloc[j] == 0:
              data_.iloc[:,data_.columns.get_loc(col) + 5] = v['場次'].iloc[j]
  return pd.concat([d_try,data_], axis = 1)

In [ ]:
def check__digit(s):
  try:
    float(s)
    return True
  except:
    return False

In [ ]:
# pd.DataFrame(d.groupby(['status', '場地狀況','名次']).count().iloc[:,0]).reset_index().merge(pd.DataFrame(d.groupby(['status', '場地狀況']).count())['名次'].reset_index().rename(columns = {'名次':'Total'}), on = ['status', '場地狀況'])

In [ ]:
# pd.DataFrame(d.groupby(['status', '場地狀況','名次']).count()).merge(pd.DataFrame(d.groupby(['status', '場地狀況']).count())['名次'].reset_index(), on = ['status', '場地狀況']).rename(columns = {'場次':'名次','名次':'Total'})

In [ ]:
# get__single__race('2020/04/26','ST','2')
# get__single__race('https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=2020/04/26&Racecourse=ST&RaceNo=2')
# continue

In [ ]:
# get__horse__pass__result('https://racing.hkjc.com/racing/information/Chinese/Horse/Performance.aspx?HorseId=HK_2018_C483')

In [ ]:
### Check the date race no is match the target or not
def check__race__no(race__date):
  url = 'https://racing.hkjc.com/racing/information/Chinese/Racing/LocalResults.aspx?RaceDate=' + race__date
  browser.get(url)
  done = False
  while not done:
    try:
      td = browser.find_element_by_class_name('js_racecard').find_element_by_tag_name('tr').find_elements_by_tag_name('td')
      done = True
    except:
      pass
      
  List = []
  for idx,i in enumerate(td):
    try:
      if idx == 1:
        l = i.find_element_by_xpath('.//*').get_attribute('src')
        List.append(l)
      else:
        l = i.find_element_by_xpath('.//*').get_attribute('href')
        List.append(l)
    except: 
      pass
  return len(List[1:-1])


In [ ]:
def get__result(date):
    result = pd.DataFrame()
    single = False
    race__place = 'HV'
    for j in range(1, 15):
      if single == False:
        try:
            result = pd.concat([result,get__single__race(date,race__place,str(j))])
            single = False
        except  :
            # print('error', j)
            race__place = 'ST'
            single = True
            pass
      if single == True:
          try:
            result = pd.concat([result,get__single__race(date,race__place,str(j))])  
          except  :
            # print('error', j)
            break
    return result, race__place


In [ ]:
for i in r[125]:
    done = False
    result, race__place = get__result(i)
    if len(result) != 0:
      while not done:
          race__no = check__race__no(i)
          max__no = max(pd.unique(result['raceno'].map(lambda x: int(x))))
          if max__no != race__no :
            print('fix')
            print(max__no, race__no)
            for j in range(int(max__no) + 1, int(race__no + 1)):
              result = pd.concat([result, get__single__race(i ,race__place, str(j))])
          else:
            display(result)
            result.to_csv('drive/My Drive/Colab Notebooks/hkjc/horse/horse__{index}.csv'.format(index = r.index(i)), index = False)
            done = True

https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=2&Racecourse=HV&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=2&Racecourse=ST&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=0&Racecourse=HV&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=0&Racecourse=ST&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=1&Racecourse=HV&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=1&Racecourse=ST&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=9&Racecourse=HV&RaceNo=1
error
error 1
https://racing.hkjc.com/racing/info/meeting/Results/Chinese/LocalResults.aspx?RaceDate=9&Racecourse=ST&RaceNo=1
error


In [ ]:
# t = get__single__race(i ,race__place, str(j))